In [1]:

from fastapi import HTTPException
from pydantic import BaseModel
from typing import List, Optional
from fastapi_offline import FastAPIOffline as FastAPT

app = FastAPT()


@app.get("/")
async def root():
    return {"message" : "Hello World"}

In [2]:
class Book(BaseModel):
    id      : int
    title   : str
    author  : str
    year    : int

books: List[Book] = []

In [3]:
import sqlite3
def get_db():
    conn = sqlite3.connect('book.db')
    conn.row_factory = sqlite3.Row
    return conn

def init_db():
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute(''' 
                    CREATE TABLE IF NOT EXISTS books(
                   id INTEGER PRIMARY KEY,
                   title TEXT NOT NULL,
                   author TEXT NOT NULL,
                   year INTEGER NOT NULL)
                   ''')
    conn.commit()
    conn.close()

@app.post("/bookdb")
async def add_db(book:Book):
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute(''' 
                INSERT INTO books
                   (id,title, author, year)  
                VALUES
                   (?,?,?,?) ''',
                   (book.id, book.title, book.author, book.year))
    conn.commit()
    conn.close()

@app.get('/bookdb')
async def get_bookdb():
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM books")
    books = cursor.fetchall()

    conn.close()
    return[
        Book (id=row['id'], title=row['title'], author=row['author'], year=row['year'])for row in books
    ] 

@app.put('/bookdb')
async def update_db(book:Book):
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute('''UPDATE books
                   SET title = ?, author=?,year=?
                   WHERE id =?''',
                   (book.title, book.author, book.year,book.id))
    conn.commit()
    conn.close()
    return {"message" : "Book has been Update!" + str(book.id), "body"    : book}

@app.delete('/bookdb')
async def delete_db(id : int):
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute('DELETE FROM books WHERE id = ?', (id,))
    conn.commit()
    if cursor.rowcount == 0:
        conn.close()
        raise HTTPException(status_code=404, detail="Book not Found")
    conn.close()
    return{"message": "BookID" +str(id)+ "is deleted"}

init_db()

In [5]:
import asyncio
import uvicorn

config = uvicorn.Config(app)
sever = uvicorn.Server(config)
await sever.serve()

INFO:     Started server process [7060]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:62114 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:62114 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:62136 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:62137 - "GET /static-offline-docs/swagger-ui-bundle.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:62136 - "GET /static-offline-docs/swagger-ui.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:62137 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:62137 - "GET /static-offline-docs/favicon.png HTTP/1.1" 200 OK
INFO:     127.0.0.1:62224 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:62236 - "POST /bookdb HTTP/1.1" 200 OK
INFO:     127.0.0.1:62241 - "DELETE /bookdb?id=1 HTTP/1.1" 200 OK
INFO:     127.0.0.1:62244 - "GET /bookdb HTTP/1.1" 200 OK
INFO:     127.0.0.1:62254 - "POST /bookdb HTTP/1.1" 200 OK
INFO:     127.0.0.1:62257 - "PUT /bookdb HTTP/1.1" 200 OK
INFO:     127.0.0.1:62265 - "PUT /bookdb HTTP/1.1" 200 OK
INFO:     127.0.0.1:62276 - "DELETE /bookdb?id=2 HTTP/1.1" 404 Not Found
INFO:     127.

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [7060]
